# League positions after every match, 1921/22-

* [EFL Regulations, 2022/23](https://www.efl.com/-more/governance/efl-rules--regulations/efl-regulations/section-3-the-league/)

* [11v11 explanation \(Click "League & table info"\)](https://www.11v11.com/league-tables/)

From 11v11:

> Historical variations in league table calculations, and treatment in the 11v11 league table generator.
>
> #### Points for a win
> Until 1981 English Football Association (FA) leagues awarded 2 points for a win.
> 
> Starting with the 1981-82 season, 3 points were awarded to encourage more competitive play.
> 
> #### Discriminating point ties
> Teams tied on points are disciminated by goal difference or goal average (depending on FA rules in force at the time), then by goals scored.
> 
> #### Goal average
> 
> From seasons 1896-97 to 1995-96, FA leagues used goal average (GAve) to resolve point ties.
> 
> Goal "average" (actually a ratio, not an average) is the result of dividing goals for by goals against (GF/GA) to three decimal places.
> 
> #### Goal average and zeros
> 
> Where GF > 0 and GA = 0 then GAve = infinity (∞), which (perhaps anomalously) gives a team with 10 goals for and 1 against a worse GAve (10/1 = 10) than a team with 1 for and 0 against (1/0 = ∞).
> 
> When GF = 0 and GA > 0, mathematically the value of GF/GA is undefined.
> 
> When GF = 0 GA = 0, mathematically the value of GF/GA is indeterminate.
> 
> #### 11v11 treatment of zeros in goal average calculations
> 
> It is unclear how the Football Leagues treated division of or by 0.
> 
> For pragmatic reasons, and in the absence of additional information, 11v11 assumes:
> 
> When GF = 0 and GA > 0, GAve = -GA (favours teams who have conceded fewer goals)
> When GF = 0 and GA = 0, GAve = 1 (consistent with GF = n and GA = n)
> Additionally:
> 
> When GF > 0 and GA = 0, GAve of ∞ is represented as 100 (cannot sort on ∞)
> 
> #### Goal difference
> 
> Goal difference (GD) is the difference between goals scored and conceded (GF-GA), and **replaced goal average in FA leagues starting with the 1976-77 season.**
> 
> #### When teams remain tied
> 
> For some leagues, if ties are not resolved according to the rules in place at the time, relative position is calculated by considering only those matches the tied teams played against each other, applying the same rules to resolve ties. Effectively these matches are treated as a sort of "mini-league".
> 
> Alternatively position may be assigned alphabetically by team name.
>
> [...]
> 
> #### Resolving points ties prior to 1896-97
> 
> There is some uncertainly regarding how points ties were resolved prior to the 1896-97 season. It may have been goal average, or possibly this situation had not previously arisen.
> 
> The 11v11 league table generator uses GAve + GF in the absence of any definitive information to the contrary.
> 
> #### Calculating position before the league ends using goal average
> 
> Using goal average makes tie resolution almost impossible for early-season tables.
> 
> This is because teams are most likely to tie when only a few games have been played, but will not usually have played the teams they tie with at that stage in the season, making it impossible to use their mutual encounters to resovle the tie.
> 
> It is noteworthy that until SSSS league tables were not published until a few weeks into the season, presumably to avoid the problems of resolving early-season points + GAve + GF ties when so few points or goals had been scored.
> 
> Users should note these points so that they are able to interpret early- and mid-season tables appropriately.

In [1]:
import pandas as pd

leagues = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-historical-league-tables/main/data/all_league_tables.csv")
leagues.head(3)

,Pos,Team,Pld,W,D,L,GF,GA,Pts,index_no,url
0,NaN,Leyton Orient,28,17,6,5,36,17,57,1,https://www.11v11.com/league-tables/league-two...
1,NaN,Stevenage,26,16,7,3,39,18,55,2,https://www.11v11.com/league-tables/league-two...
2,NaN,Northampton Town,27,14,7,6,42,28,49,3,https://www.11v11.com/league-tables/league-two...


In [2]:
leagues.columns = leagues.columns.str.lower()
leagues.url = leagues.url.str.replace("/\n", "", regex=False)
leagues['date'] = leagues.url.str.split("/", regex=False).str[5].str.replace("-", " ")
leagues.date = pd.to_datetime(leagues.date)
leagues['gd'] = leagues.gf - leagues.ga
leagues['g_av'] = leagues.gf / leagues.ga
leagues['pts_and_goals'] = leagues[["pts", "gd", "gf"]].apply(tuple, axis=1)
leagues['ranking'] = leagues.groupby("url").pts_and_goals.rank("min", ascending=False)

leagues = leagues[["date", "pos", "ranking", "team", "pld", "w", "d", "l", "gf", "ga", "gd", "g_av", "pts"]]
leagues.head(3)

,date,pos,ranking,team,pld,w,d,l,gf,ga,gd,g_av,pts
0,2023-01-28,NaN,1.0,Leyton Orient,28,17,6,5,36,17,19,2.117647,57
1,2023-01-28,NaN,2.0,Stevenage,26,16,7,3,39,18,21,2.166667,55
2,2023-01-28,NaN,3.0,Northampton Town,27,14,7,6,42,28,14,1.500000,49


In [3]:
trfc = leagues[leagues.team == 'Tranmere Rovers'].drop(["pos", "team"], axis=1).rename(columns = {"date": "game_date"})
trfc.head(3)

,game_date,ranking,pld,w,d,l,gf,ga,gd,g_av,pts
12,2023-01-28,13.0,27,10,8,9,29,23,6,1.260870,38
38,2023-01-14,15.0,26,9,8,9,28,23,5,1.217391,35
62,2023-01-07,15.0,25,9,7,9,27,22,5,1.227273,34


In [4]:
results = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", parse_dates=["game_date"])
results = results.merge(trfc, on="game_date")

In [5]:
results = results.drop(['home_team',
    'away_team',
    'home_goals',
    'away_goals',
    'secondary_score',
    'source_url',
    'stadium',
    'game_type'], axis=1)

# CSV only required if using expanded dataset (manager, final record, etc.)
# results.to_csv("../data/results_expanded.csv", index=False)

In [6]:
results_mini = results[['season', 'competition', 'league_tier', 'ssn_comp_game_no', 'ranking', 'pld', 'pts', 'manager']]

In [7]:
# Extra step to fix situations where Tranmere's final league game was played before other teams'. League position for the final game of the season will now be the final league position, not the position at EOD of the game.

def get_season(url):
    year_2 = url[-5:-1]
    year_1 = int(year_2) - 1
    year_2 = str(year_2)[-2:]
    season = f"{year_1}/{year_2}"
    return season

finishes = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-final-league-position/main/data/final-positions.csv")

finishes["season"] = finishes.url.apply(get_season)

finishes = finishes.query("Team == 'Tranmere Rovers'").groupby("season").max().reset_index()[["season", "Pld", "index_no"]].rename(columns={"Season": "season", "Pld": "pld", "index_no": "final_pos"})

results_mini = results_mini.merge(finishes, on=["season", "pld"], how="outer")
results_mini["ranking"] = results_mini.apply(lambda x: x["final_pos"] if pd.notnull(x["final_pos"]) else x["ranking"], axis=1)
results_mini = results_mini.drop(columns=["final_pos"])

In [8]:
results_mini.to_csv("../docs/input/results_mini.csv", index=False)